In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
import os

In [2]:
#load data
df = pd.read_csv("/home/julian/Project/code/quora_duplicate_questions.tsv",delimiter='\t')

names=df.columns.values
print(names)
print(df.shape)

y=df['is_duplicate']
print(y.shape)

y_train=y[0:100000]
y_test=y[100000:200000]

subset=df[0:200000]
#df=subset

#print(df)

['id' 'qid1' 'qid2' 'question1' 'question2' 'is_duplicate']
(404290, 6)
(404290,)


In [3]:
#training for embedding
import gensim


In [4]:

df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))


In [5]:
questions=list(df['question1'])+list(df['question2'])
c=0
for i in tqdm(questions):
    questions[c]=list(gensim.utils.tokenize(i,deacc=True,lower=True))
    c+=1

100%|██████████| 808580/808580 [00:46<00:00, 17260.89it/s]


In [9]:
from gensim.models import Word2Vec
embed_model=Word2Vec(questions,size=10,min_count=1)
print(embed_model['what'])

[ 2.77741027 -0.23460764 -2.06365085  2.37654829 -0.06589089 -0.1005596
 -0.29259163 -0.94224375 -1.03192949  1.60735917]


In [7]:
import tensorflow as tf

In [8]:
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence,text
from keras.layers import Dense, Input, LSTM, Embedding, Dropout,Merge

Using TensorFlow backend.


In [10]:

tk=text.Tokenizer(filters="")
max_len=40
tk.fit_on_texts(list(subset.question1.values)+list(subset.question2.values))
x1 = tk.texts_to_sequences(subset.question1.values)
x1 = sequence.pad_sequences(x1,maxlen = max_len)

x2 = tk.texts_to_sequences(subset.question2.values)
x2 = sequence.pad_sequences(x1,maxlen = max_len)

In [11]:
q=subset['question1']
print(q[0])
print(x1[0])

What is the step by step guide to invest in share market in india?
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    2    3    1 1398
   54 1398 3004    7  478    8  766  573    8   58]


In [12]:
nb_words=len(tk.word_index)+1
print(nb_words)
#print(subset.question1.values)

136893


In [13]:
x1_train=x1[0:100000]
x1_test=x1[100000:200000]
x2_train=x2[0:100000]
x2_test=x2[100000:200000]

In [14]:

print(x1_train.shape)
print(x1_test.shape)
print(y_train.shape)
print(y_test.shape)
#y_train=y_train.reshape((1,10))
#y_test=y_test.reshape((1,10))

(100000, 40)
(100000, 40)
(100000,)
(100000,)


In [15]:


embedding_matrix = np.zeros((nb_words, 10))

for word, i in tk.word_index.items():
    embedding_vector = None 
    if word in embed_model.wv.vocab:
        embedding_vector = embed_model[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    
#print(embedding_matrix)

In [16]:
embedding_vecor_length = 10
model1 = Sequential()
model1.add(Embedding(nb_words, embedding_vecor_length,weights=[embedding_matrix], input_length=max_len))
x1=model1.add(LSTM(100,return_sequences=True))
#x1=model1.add(LSTM(100))
#model1.add(Dense(1, activation='sigmoid'))
#model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())
#model1.fit(x1_train, y_train, validation_data=(x1_test, y_test),epochs=2,batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 10)            1368930   
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 100)           44400     
Total params: 1,413,330
Trainable params: 1,413,330
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
embedding_vecor_length = 10
model2 = Sequential()
model2.add(Embedding(nb_words, embedding_vecor_length,weights=[embedding_matrix], input_length=max_len))
x2=model2.add(LSTM(100,return_sequences=True))
#x2=model2.add(LSTM(100))
#model2.add(Dense(1, activation='sigmoid'))
#model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
#model2.fit(x2_train, y_train, validation_data=(x2_test, y_test),epochs=2,batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 10)            1368930   
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 100)           44400     
Total params: 1,413,330
Trainable params: 1,413,330
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
model = Sequential()
model.add(Merge([model1, model2], mode='concat'))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit([x1_train,x2_train],y_train,validation_data=([x1_test,x2_test],y_test),epochs=1,batch_size=100)

/home/julian/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_3 (Merge)              (None, 40, 200)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 2,967,361
Trainable params: 2,967,361
Non-trainable params: 0
_________________________________________________________________
None
Train on 100000 samples, validate on 100000 samples
Epoch 1/1
 99900/100000 [============================>.] - ETA: 2s - loss: 


KeyboardInterrupt



In [22]:
scores = model.evaluate([x1_test,x2_test],y_test,verbose=0)
print(scores)

KeyboardInterrupt: 

In [18]:
y_pred=model.predict([x1_test,x2_test],batch_size=2,verbose=0)
print(y_pred,y_test)

(array([[ 0.4801172 ],
       [ 0.48014233],
       [ 0.48034409],
       [ 0.4797821 ],
       [ 0.47950292],
       [ 0.47917807],
       [ 0.47989726],
       [ 0.48013282],
       [ 0.48013255],
       [ 0.47951597]], dtype=float32), 10    0
11    1
12    1
13    1
14    0
15    1
16    1
17    0
18    1
19    0
Name: is_duplicate, dtype: int64)


In [33]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(x1,x2)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [52]:
embedding_layer = Embedding(nb_words,
                                10,
                                weights=[embedding_matrix],
                                input_length=30,
                                trainable=False)
print(embedding_layer)
lstm_layer = LSTM(75, recurrent_dropout=0.2)

NameError: name 'embedding_matrix' is not defined

In [27]:
MAX_SEQUENCE_LENGTH=30
sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

In [33]:
HIDDEN_DIM=30
EMBED_DIM=10
seq_maxlen=30
ques1_enc = Sequential()
ques1_enc.add(Embedding(output_dim=EMBED_DIM, input_dim=nb_words,
                       weights=[embedding_matrix], mask_zero=True))
ques1_enc.add(LSTM(HIDDEN_DIM, input_shape=(EMBED_DIM, seq_maxlen), 
                       return_sequences=False))
ques1_enc.add(Dropout(0.3))
    
ques2_enc = Sequential()
ques2_enc.add(Embedding(output_dim=EMBED_DIM, input_dim=nb_words,
                       weights=[embedding_matrix], mask_zero=True))
ques2_enc.add(LSTM(HIDDEN_DIM, input_shape=(EMBED_DIM, seq_maxlen), 
                       return_sequences=False))
ques2_enc.add(Dropout(0.3))

In [34]:
model = Sequential()
model.add(Merge([ques1_enc, ques2_enc], mode="sum"))
model.add(Dense(2, activation="softmax"))
    
model.compile(optimizer="adam", loss="categorical_crossentropy",
                  metrics=["accuracy"])
    

/home/julian/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [ ]:
checkpoint = ModelCheckpoint(
        filepath=os.path.join(MODEL_DIR, "quora_dul_best_lstm.hdf5"),
        verbose=1, save_best_only=True)
    model.fit([x_ques1train, x_ques2train], ytrain, batch_size=BATCH_SIZE,
              epochs=NBR_EPOCHS, 
              validation_split=0.1,
              verbose=2,
              callbacks=[checkpoint])
    

In [58]:
#model.fit([data_1, data_2], y_train, batch_size=2,epochs=1)
print(y_train)
model.fit(embedding_matrix,y_train,batch_size=2,epochs=1)  

[0 0 0 0 0 1 0 1 0 0]


ValueError: The model expects 2 input arrays, but only received one array. Found: array with shape (122, 10)